# Coletar Informações Gerais dos Jogos Steam

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import time

import requests
import json
import html


In [ ]:
all_ids = []
for year in range(2019, 2025):
    ids_year = pd.read_csv(f"data/most_played_{year}.csv").to_dict('records')
    all_ids.append(ids_year)


id_all_games = set()
for ids_year in all_ids:
    for id in ids_year:
        id_all_games.add((id['id']))

id_all_games = list(id_all_games)
print(id_all_games)

[629760, 1551360, 427520, 1716740, 1145350, 230410, 1506830, 1222670, 1203220, 250900, 252950, 526870, 1222680, 578080, 550, 632360, 892970, 1196590, 552500, 1282100, 201270, 570, 601150, 242760, 252490, 779340, 755790, 2399830, 2050650, 648800, 1250410, 812140, 1623660, 2881650, 671860, 394360, 1794680, 359550, 1818750, 105600, 1966720, 1790600, 594570, 1172620, 920210, 1151640, 1174180, 1158310, 1938090, 2507950, 251570, 1623730, 2767030, 899770, 292030, 2429640, 945360, 730, 1085660, 2183900, 8930, 397540, 271590, 1254120, 1517290, 532210, 677620, 2073850, 952060, 2669320, 381210, 322330, 1238810, 2252570, 1260320, 304930, 740130, 444200, 2074920, 39210, 924970, 814380, 1332010, 289070, 739630, 1063730, 2195250, 1238840, 1643320, 1361210, 2054970, 1811260, 1454400, 2139460, 1089350, 377160, 617290, 1568590, 431960, 976730, 2694490, 834910, 236390, 489830, 534380, 1599340, 1868140, 238960, 49520, 2933620, 1240440, 582010, 1449850, 1446780, 553850, 881020, 990080, 1248130, 281990, 141

In [3]:
info_games = []
for id in id_all_games:
    id = str(id)
    game_info_url = f"https://store.steampowered.com/api/appdetails?"
    params = {
        'appids': id,
        'cc': 'us',
        'l': 'english',
        'success': '1'
    }
        
    request = requests.get(game_info_url, params=params, timeout=10)
    time.sleep(1)
    
    game_data = json.loads(request.text)
    
    if game_data:
        try:
            data = game_data[id]['data']
            # coletando: nome, data de lançamento, idade mínima, se é gratuito, tipo da moeda e preço.
            # caso queira coletar mais informações, atribuir variáveis abaixo.
            name = data['name']
            print(name)

            release_date = data['release_date']['date']
            
            genres_list = data['genres']
            genres = ', '.join([g['description'] for g in genres_list]) if genres_list else None
            
            try:
                requirements = data['pc_requirements']['minimum']
                decoded_html = requirements.encode().decode('unicode_escape')
                decoded_html = html.unescape(decoded_html)
                soup = BeautifulSoup(decoded_html, "html.parser")
                
                graphic_list = soup.find_all('li')
                for item in graphic_list:
                    if ':' in item.get_text():
                        key, value = item.get_text().split(":", 1)
                        if key.strip().lower() == 'graphics':         
                            minimum = value.strip()
                        
            except Exception:
                minimum = None
                
            # Se existir preco, atribua a variável preço
            # Caso contrário, atrivua a variável preço None  
            try:
                preco = float(data['price_overview']['initial']) / 100
            except Exception:
                preco = 0

            # Criando dicionário do jogo atual e adicionando na lista
            info_games.append({
                'id' : id, 
                'game' : name, 
                'release_date' : release_date, 
                'price_USD' : preco,
                'genres' : genres,
                'pc_requirements' : minimum
                })
        except Exception:
            print(f"Erro ao processar ID {id}.")
            
df = pd.DataFrame(info_games)
df.to_csv("data/all_games_info.csv", index=False, encoding="utf-8")
print("ARQUIVO all_games_info.csv CRIADO.")

MORDHAU
Forza Horizon 5
Factorio
Starfield
Hades II
Warframe
FIFA 22
The Sims™ 4
NARAKA: BLADEPOINT
The Binding of Isaac: Rebirth
Rocket League®
Satisfactory
Need for Speed™ Heat
PUBG: BATTLEGROUNDS
Left 4 Dead 2
Risk of Rain 2
Valheim
Resident Evil Village
Warhammer: Vermintide 2
REMNANT II®
Total War: SHOGUN 2
Dota 2
Devil May Cry 5
The Forest
Rust
Total War: THREE KINGDOMS
Ring of Elysium
ARK: Survival Ascended
Resident Evil 4
Raft
Microsoft Flight Simulator (2020) 40th Anniversary Edition
Assassin's Creed® Odyssey
MIR4
Content Warning
BattleBit Remastered
Hearts of Iron IV
Vampire Survivors
Tom Clancy's Rainbow Six® Siege X
MultiVersus
Terraria
Lethal Company
DRAGON BALL: Sparking! ZERO
Total War: WARHAMMER II
Sea of Thieves: 2025 Edition
LEGO® Star Wars™: The Skywalker Saga
Horizon Zero Dawn™ Complete Edition
Red Dead Redemption 2
Crusader Kings III
Call of Duty®
Delta Force
7 Days to Die
Palworld
Marvel Rivals
Last Epoch
The Witcher 3: Wild Hunt
THRONE AND LIBERTY
Among Us
Counte

## Classificar GPUs

In [5]:
gpu_classification = {
    "very low": [
        "Intel GMA X4500, NVIDIA GeForce 9600M GT, AMD/ATI Mobility Radeon HD 3650 - requires 256MB VRAM and DirectXÂ® 11",
        "Integrated Graphics",
        "128 MB of Video RAM and support for Pixel Shader 3.0. Supported Video Cards: NVIDIA GeForce 6600 or better, ATI Radeon X1300 or better, Intel GMA X4500 or better",
        "NVIDIA Geforce GTS 450 / AMD Radeon HD 5570",
        "NVIDIA GeForce 8600/9600GT, ATI/AMD Radeon HD2600/3600",
        "NVIDIA GeForce GT 730 or equivalent",
        "1 GB & AMD 5570 or nVidia 450 or Intel Integrated Graphics 530",
        "Video card with 128 MB, Shader model 2.0. ATI X800, NVidia 6600 or better",
        "DirectX 9 graphics card with 512Mb Video RAM: AMD Radeon HD 2600 XT, nVidia 8600"
    ],
    "low": [
        "Nvidia GeForce GTX 650 Ti / AMD Radeon HD 7850",
        "Nvidia Geforce GTX 650, AMD Radeon HD 7770 graphics card or better (min. 2 GB VRAM, DX11 support)",
        "Nvidia GeForce GTX 650 2GB or AMD Radeon HD 7770 2GB",
        "Nvidia Geforce GTX 660 / AMD HD 7870 / Intel Arc A380",
        "NVIDIA GeForce GTX 750 Ti (Maxwell or newer), 2 GB or AMD Radeon R7 360, 2 GB",
        "AMD Radeon R9 285, NVIDIA GeForce GTX 660",
        "NVIDIA GeForce GTX 460 / AMD HD6850",
        "NVIDIA GeForce GTX 760 or AMD R9 270X",
        "NVIDIA GeForce GTX 760 | AMD Radeon HD 7950",
        "NVIDIA GeForce GTX 460 or AMD Radeon HD 5870",
        "NVIDIAÂ® GeForceÂ® GTX 660 2GB or GTX 1050 2GB / AMD Radeon HD 7850 2GB",
        "NVIDIA GeForce GTX 660, ATI Radeon HD 7850",
        "AMD Radeon R9 285, NVIDIA GeForce GTX 660 (2GB VRAM with Shader Model 5.0) (MORE DETAILS HERE)",
        "NVIDIA GeForce GTX 560 Ti / AMD Radeon HD 6850",
        "DirectX 11 level video card: AMD Radeon 77XX / NVIDIA GeForce GTX 660. The minimum supported resolution for the game is 720p.",
        "NVIDIA GeForce GTX 660 1GB",
        "GTX 580 / AMD HD 7870",
        "NVIDIA GTX 660 2GB or AMD Radeon HD 7850 2GB"
    ],
    "medium": [
        "NVIDIA GeForce GTX 1050 ti or AMD R9 380",
        "Nvidia GTX 1650 4GB / AMD RX 5500XT 4GB / Intel ARC A380 6GB",
        "NVIDIA GeForce GTX 1650, 4 GB or AMD Radeon RX 480, 4 GB",
        "NVIDIA GTX 1650 Super or AMD Radeon RX 5500 XT, with a minimum of 4GB of VRAM",
        "NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 470",
        "NVIDIA GeForce GTX 1050Ti (4GB), GTX 1060 (3GB), GTX 1650 (4GB) or AMD Radeon R9 280(3GB), AMD Radeon R9 290 (4GB), RX 470 (4GB)",
        "Nvidia GeForce GTX 980 / AMD Radeon RX 590 / Intel Arc A750",
        "NVIDIA GeForce GTX 1050 Ti 4GB or AMD Radeon RX 570 4GB",
        "AMD Radeon RX 470",
        "NVIDIA GeForce GTX 1060 3GB or AMD Radeon RX 570 4GB",
        "NVIDIA GeForce 760 GTX or AMD Radeon R9 280",
        "AMD Radeon RX 560 with 4GB VRAM / NVIDIA GeForce GTX 1050 Ti with 4GB VRAM",
        "Nvidia GeForce GTX 1060 6GB / AMD Radeon RX 580 8GB / Intel Arc A750",
        "Nvidia GeForce GTX 780 (3 GB) or AMD Radeon R9 290 (4GB)",
        "GeForce GTX 1050Ti / AMD Radeon RX 570 / Intel Arc A380",
        "NVIDIA GTX 770 | AMD Radeon RX 570",
        "NVIDIA GeForce GTX 960 2GB / AMD Radeon R7 370 2GB",
        "NVIDIA GeForce GTX 970 or higher, AMD Radeon RX 480 or higher",
        "NVIDIA GeForce GTX 1050 Ti or AMD Radeon RX 580",
        "NVIDIA GeForce GTX 960 4GB or AMD Radeon RX 470 4GB",
        "Nvidia GeForce GTX 1050Ti, AMD Radeon RX 470 graphics card or better (min 3GB VRAM, DX12 support)",
        "NVIDIA GeForce GTX 1060 (req. 6GB VRAM) / AMD Radeon RX 580 (req. 6GB VRAM) / Intel ARC A380",
        "AMD Radeon RX 5600 XT, NVIDIA GeForce 1080",
        "AMD Radeon RX 560,Nvidia GeForce GTX 1050 Ti",
        "NVIDIA GeForce GTX 970 OR AMD Radeon RX 570",
        "NVIDIA GeForce GTX 1650",
        "NVIDIA GeForce GTX 1060 6GB / AMD Radeon RX 580 8GB",
        "NVIDIA GeForce GTX 1050 Ti OR AMD Radeon RX 570"
    ],
    "high": [
        "NVIDIA GTX 1650 Super or AMD Radeon RX 5500 XT, with a minimum of 4GB of VRAM",
        "NVIDIA GeForce GTX 1070 / AMD Radeon RX 5500 XT with 8GB VRAM",
        "GeForce GTX 1650  / AMD Radeon RX 590",
        "NVIDIA GTX 1060 3 GB / AMD Radeon RX 590"
    ],
    "very high": [
        "AMD Radeon RX 5700, NVIDIA GeForce 1070 Ti"
    ]
}

df_game_info = pd.read_csv("data/all_games_info.csv")
gpus_class = []
for gpu in df_game_info['pc_requirements']:
    for cls, gpus in gpu_classification.items():
        if gpu in gpus:
            gpus_class.append({"gpu": gpu, "classification": cls})
            break
        
df_gpu_class = pd.DataFrame(gpus_class)
df_gpu_class.to_csv("data/gpu_classification.csv", index=False)